# Order Updater 

Set the PM Instruction to "HOLD". If the quantity is above 100,000, set the PM Instruction to "SPEAK TO PM"

In [1]:

import sys
import datetime
import uuid,datetime,requests,pprint,math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
today = datetime.datetime.now().isoformat()[:10]
from ipywidgets import widgets
sys.path.append('../modules')
from spc_config import *
def generate_headers():
    headers = {
        'VND.com.blackrock.API-Key': APIkey,
        'VND.com.blackrock.Origin-Timestamp': str(datetime.datetime.utcnow().replace(microsecond=0).astimezone().isoformat()),
        'VND.com.blackrock.Request-ID': str(uuid.uuid1())
    }
    return headers


#Create a field
pflio = widgets.Text(description = "Portfolio")
pflio.value = "BRS-EQ-EM"
display(pflio)

Text(value='BRS-EQ-EM', description='Portfolio')

In [5]:
print(f"Step 1: Retrieving order details for {pflio.value}\n")

# api-endpoint 
URL = f"https://{client_env}.blackrock.com/api/trading/order-management/order/v1/orders:filter"


# include parameters and headers

payload = {
      "query": {
     "portfolioGroupCriterion": {
      "orderStatuses": [
        "Open", "Authorized"
      ],
       "portfolioGroupTicker": pflio.value
    }
}
}
        


r = requests.post(URL,auth=(username,pwd),json=payload,headers=generate_headers())

# extracting data in json format 
data = r.json() 
columnOrder = ['id','assetId', 'orderDate','settleDate','transactionType','orderStatus','pmInitials','traderInitials', 'marketPrice']

#printing the output 
df = pd.DataFrame(data['orders'])
df = df[columnOrder]
df = df[df['orderStatus'] == "Open"]
print(f"Found {len(df)} open orders \n")
df

Step 1: Retrieving order details for INTERNGRP4

Found 0 open orders 



,id,assetId,orderDate,settleDate,transactionType,orderStatus,pmInitials,traderInitials,marketPrice


In [8]:
print("Step 3: update orders")

orders = data["orders"]
orders[:] = [order for order in orders if order["orderStatus"] == "Open"]

for order in orders:
    orderDetail = order["orderDetails"][0]
    order['pmInstruction'] = "HOLD"
    if orderDetail["quantity"] > 100000:
        order['pmInstruction'] = "SPEAK TO PM"
        
#print(orders)

Step 3: update orders


In [9]:
removeOrder = ["createdTimestampAtz","factor","factorDate","interest","modifiedBy","modifiedTimestampUtc","orderStatus","principal","touchCount","version","yield","comments"]
removeDetail = ["modifiedBy","modifiedTimestampUtc","orderReservations","quantityBooked"]
for order in orders:
    for k in removeOrder:
        order.pop(k,None)
    for detail in order["orderDetails"]:
        for k in removeDetail:
            detail.pop(k,None)

In [10]:
print("Step 4: Send updated orders")

# api-endpoint 
URL = F'https://{client_env}.blackrock.com/api/trading/order-management/order/v1/orders:batchPost'

payload_req = {
  "options": {
    "complianceTimeout": 0,
    "extEntityType": "string",
    "orgId": 0,
    "runCompliance": False,
    "whatIf": False
  },
  "orders": orders
}

# sending post request and saving the response as response object 
r = requests.post(URL,auth=(username,pwd), json = payload_req, headers = headers)


# extracting data in json format 
data_res = r.json()

# printing the output 
#display(data_res)
print("\nFinished !")

Step 4: Send updated orders


NameError: name 'headers' is not defined